In [1]:
###########LIBRARY CELL###########
%load_ext watermark
%watermark -v -m -p numpy,scipy,pandas,matplotlib,sklearn -g 

CPython 3.6.7
IPython 7.4.0

numpy 1.16.2
scipy 1.2.1
pandas 0.24.2
matplotlib 3.0.3
sklearn 0.20.3

compiler   : GCC 8.2.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   : be59a2b17efe898dacde8a9aa535c07ad30e093f


In [2]:
###########LIBRARY CELL###########
from __future__ import division, print_function
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
import math
import datetime
import re
import sys

In [3]:
###########VERSION OF COMPARATOR FOR SD<->SD and SD<->TDC
###########LIBRARY CELL###########
#########CONSTANTS################
COMTYPE_SDSD, COMTYPE_SDTDC, COMTYPE_TDCSD, COMTYPE_TDCTDC = 1,2,3,4;
ENG,DE = 1,2
comparison_type_text = {
    COMTYPE_SDSD:"SimadynD <=> SimadynD",
    COMTYPE_SDTDC:"SimadynD <=> TDC",
    COMTYPE_TDCSD:"TDC <=> SimadynD", #TODO, not yet supported
    COMTYPE_TDCTDC:"TDC <=> TDC" #TODO, not yet supported
    }
simadyn_lang_text = {
    ENG:"English",
    DE:"Deutsch"
    }
COMPARATOR_AUTHOR = "Anton Tushev" 
COMPARATOR_VERSION = "v.40.CMB"

In [4]:
###########LIBRARY CELL###########
#########CONFIGURATION################
COMPARISON_TYPE = COMTYPE_SDTDC # COMTYPE_SDSD,COMTYPE_SDTDC, 
SIMADYND_LANG = DE #ENG, DE 
CSV_DELIMITER = "," #CAN BE ; or ,

PATH_TO_DATA = '/notebooks/pinda/data'
###PATH #1
#PATH_TO_STRUCG_1 = '/notebooks/pinda/data/eko/test_1'
#PATH_TO_STRUCG_1 = '/notebooks/pinda/data/eko/test_1'  #' ##debug
PATH_TO_STRUCG_1 = '/notebooks/pinda/data/eko/200117_after_tool' 
PATH_TO_TDC_DATA_1 = '' #AMEH_SX01_PS_20200130.csv
###PATH #2
#PATH_TO_STRUCG_2 = '/notebooks/pinda/data/eko/test_2'
PATH_TO_STRUCG_2 = ''
PATH_TO_TDC_DATA_2 = 'AMEH_SX01_PS_20200304.csv'

PATH_TO_REPORTS = '/notebooks/pinda/reports'
FILE_1_NAME_TEMP1 = '@source1_progress_bkup_1.csv'
FILE_1_NAME_TEMP2 = '@source1_progress_bkup_2.csv'
FILE_1_NAME_TEMP3 = '@source1_progress_bkup_3.csv'
FILE_2_NAME_TEMP1 = '@source2_progress_bkup_1.csv'
FILE_2_NAME_TEMP2 = '@source2_progress_bkup_2.csv'
FILE_2_NAME_TEMP3 = '@source2_progress_bkup_3.csv'

#SD parsing parameters
SD_RACK_NAMES = ['sx01']
#SD_RACK_NAMES = ['tx01']
IGNORE_CHART = ['@SIMD']

#report parameters
#this comparator works with following data types and connectors:
filt_type_base = ['B1','TF','N2','N4','V2','O2','V4','NF','V1','I2','TR','RR']  #,'NS'
filt_inout = ['<']#,'-','=', '>'

###TODO: outputs '>' for sd-tdc comparison doesn't work.

COMPARATOR_README = "===================================================================================\
\r\nAdvanced comparator SimadynD and TDC, software version: %s \r\nAuthor: %s \
\r\nComparison type: %s\
\r\nComparison source #1: %s\
\r\nComparison source #2: %s\
\r\nList of SimadynD subfolders for parsing is %s \
\r\nList of ignored charts is %s \
\r\n===================================================================================\
"%(
    COMPARATOR_VERSION,
    COMPARATOR_AUTHOR,
    comparison_type_text[COMPARISON_TYPE],
    PATH_TO_STRUCG_1 if (COMPARISON_TYPE==COMTYPE_SDSD)|(COMPARISON_TYPE==COMTYPE_SDTDC) else PATH_TO_TDC_DATA_1,
    PATH_TO_STRUCG_2 if (COMPARISON_TYPE==COMTYPE_SDSD)|(COMPARISON_TYPE==COMTYPE_TDCSD) else PATH_TO_TDC_DATA_2,
    SD_RACK_NAMES,
    IGNORE_CHART)
print(COMPARATOR_README)

Advanced comparator SimadynD and TDC, software version: v.40.CMB 
Author: Anton Tushev 
Comparison type: SimadynD <=> TDC
Comparison source #1: /notebooks/pinda/data/eko/200117_after_tool
Comparison source #2: AMEH_SX01_PS_20200304.csv
List of SimadynD subfolders for parsing is ['sx01'] 
List of ignored charts is ['@SIMD'] 


In [7]:
###########LIBRARY CELL###########
#Generate Simadyn chart structure
def sd_get_chart_list(path_to_struc_g,sd_rack_names):
    #filter chart files
    n = re.compile("^(((?!old).)*).((cfp)|(ofp))$")
    file_list=[] 
    file_list = {rack_name:[item for item in os.listdir(os.path.join(path_to_struc_g,rack_name)) \
        # chart file names filter conditions
        if os.path.isfile(os.path.join(path_to_struc_g,rack_name, item)) and (n.search(item)) and len(item)<11 \
        ] for rack_name in sd_rack_names}
    file_list_filtered = dict(file_list)
    for i in file_list:
        for k in file_list[i]:
            chart_name = k.split('.')[0]
            if "%s.cfp"%chart_name in file_list[i]:
                try:
                    file_list_filtered[i].remove("%s.ofp"%chart_name)
                except ValueError:
                    pass  # do nothing
    out_dict = dict()
    for i in file_list_filtered:
        out_dict[i] = dict()
        for k in file_list_filtered[i]:
            out_dict[i][k.split('.')[0]]=os.path.join(path_to_struc_g,i,k)
    if out_dict:
        return out_dict
    else:
        return -1
    
#Generate Simadyn cpu mapping structure
def sd_get_cpu_charts_list(path_to_struc_g,sd_rack_names):
    #filter chart files
    n = re.compile("^((\d+).*)\.(mpn)$")
    file_list=[] 
    file_list=[] 
    file_list = {rack_name:[item for item in os.listdir(os.path.join(path_to_struc_g,rack_name)) \
        # chart file names filter conditions
        if os.path.isfile(os.path.join(path_to_struc_g,rack_name, item)) and (n.search(item)) and len(item)<11 \
        ] for rack_name in sd_rack_names}
    out_dict = dict()
    for i in file_list:
        out_dict[i] = dict()
        for k in file_list[i]:
            out_dict[i][k.split('.')[0]]=os.path.join(path_to_struc_g,i,k)
    if out_dict:
        return out_dict
    else:
        return -1

def get_link_source(x):
    regex = re.compile(r"\s*L.\$.+")
    link_source_list  = list(filter(regex.search, x['link'].split('"')))
    return [(k.split("$")[1], x["chart_block_signal"].replace("/", "\\")) for k in link_source_list]  

def get_linked_value(x,link_map):
    if pd.isna(x['link']):
        x['value_linked'] = x['value'];
    else:
        if x['out']:
            x['value_linked'] = x['link'].split('"')[0]
        else:
            regex = re.compile(r"\s*\w?\w\$.+")
            raw_link = x['link'].split('"')[0]
            if regex.match(raw_link):
                pure_link = raw_link.split("$")[1]
                if pure_link in link_map:
                    x['value_linked'] = link_map[pure_link]
                else:
                    x['value_linked'] = raw_link
            else:
                x['value_linked'] = raw_link
    return x

def parse_tdc_source_file(path_to_data,filename):
    data_tdc = pd.read_csv(os.path.join(path_to_data,filename),keep_default_na=False,na_values=[''], encoding="cp1251",sep=CSV_DELIMITER,header=0)
    data_tdc = data_tdc[data_tdc.columns[[0,1,2,3,5,7,14,15]]]
    data_tdc.columns = ['rack_cpu','chart','block','signal','value','link','type','out']
    data_tdc = pd.concat([data_tdc, data_tdc['rack_cpu'].str.split("\\",expand = True)[[0,1]]], axis=1) 
    data_tdc.columns = ['rack_cpu','chart','block','signal','value','link','type','out','rack','cpu']
    data_tdc["out"]= data_tdc["out"].replace(("IN","OUT"),(0,1)).astype('bool') 
    data_tdc = data_tdc[~data_tdc['chart'].isin(IGNORE_CHART)]
    data_tdc["block"] = data_tdc["block"].astype(str)
    data_tdc["chartblock"] = data_tdc["chart"].str.upper()+"/"+data_tdc["block"].str.upper()
    data_tdc["chart_block_signal"] = data_tdc["chart"].str.upper()+"/"+data_tdc["block"].str.upper()+"."+data_tdc["signal"].str.upper()
    data_tdc['value_linked'] = None
    link_source = data_tdc[~data_tdc['link'].isna()].apply(get_link_source,axis = 1)
    link_source = dict([item for sublist in link_source for item in sublist])
    data_tdc = data_tdc.apply(get_linked_value,axis = 1,link_map=link_source) #, 
    return data_tdc


In [8]:
###########LIBRARY CELL###########
#########################SimadynD row data generating library#################
NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE,CPU,CPU_SOURCE,CPU_LINK,CHART,BLOCK,LINKS  = 0,1,2,3,4,5,6,7,8,9,10

#open chart for reading
def sd_open_chart(chart_addr):
    try:
        with open(chart_addr) as f:
            lines = [line.rstrip('\n') for line in f]
    except EnvironmentError: # parent of IOError, OSError *and* WindowsError where available
        #print('ERROR File not exists!')
        return -1
    return lines

#generate chart mapping: line {block name:[block line begin, block line end]}
def sd_get_chart_map(inp_line):
    b = re.compile(r"^\s*\d+ ?([a-zA-Z0-9_]+) +: [a-zA-Z0-9_.@]+\s*,\s*POS=")
    el = re.compile(r'^ *\d+\s+[+]')
    #cp = re.compile(r'\(([a-zA-Z0-9_-]+).([a-zA-Z0-9_-]+)\s+\S+\s+\S+\)') #not used, because not all charts consist
    block_list = []
    mark_bloc = False
    #mark_cpu = False
    out_dict = dict()
    line_counter = 0
    for k in inp_line:
        #Find rack name and cpu name, e.g. MP-TX04 PN-D05_P2
        #if not mark_cpu:
        #    cp_s = cp.search(k)
        #    if cp_s :
        #        rack,cpu = cp_s.group(1),cp_s.group(2)
        #        mark_cpu = True
        b_s = b.search(k)
        if b_s :
            mark_bloc = True
            curr_block_name = b_s.group(1)
            block_list.append(curr_block_name)
            mark_bloc_line_beg = line_counter
        elif el.search(k):
            if mark_bloc:
                mark_bloc = False
                out_dict[curr_block_name] = [mark_bloc_line_beg,line_counter]
        line_counter = line_counter+1        
    #print(len(block_list))
    if(out_dict):
        return out_dict
    else:
        return -1

#check if line is signal or link
def sd_check_line_signal(line):
    sf = re.compile(r"^\s*\d+\s+(\w+)\s+(\w+)\s+([<>=-])")
    sa = re.compile(r"^\s*&+\s+(\(.*\))\s*$")
    if sf.search(line):
        out = 1 #signal
    elif sa.search(line):
        out = 2 #link
    else:
        out = 0 #nothing
    return out



#exstract link from like
def sd_get_line_link(line): 
    m = re.compile(r"^\s*&+\s+\((.*)\)\s*$")
    n = m.search(line)     
    return n.group(1)  

#convert list to pandas data frame
def sd_conv_block_data_to_df(block_data):
    #list element of row signal data:[[NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE],[links]]
    block_df = pd.DataFrame([[j for j in k[0]]+[k[1]] for k in block_data])
    return block_df
#convert list to better representation
def sd_conv_block_data_to_list(block_data):
    #list element of row signal data:[[NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE,CPU,CPU_SOURCE,CPU_LINK,CHART,BLOCK],[links]]
    block_data_list = [[j for j in k[0]]+[k[1]] for k in block_data]
    return block_data_list

#parsing mapping file
#generate cpu,link db by cpu mapping file
def sd_build_cpu_map(sd_chart_list):
    data_link_map_list = []
    bg_lang_options = {ENG:"Connector list of the function packets",
                       DE:"Konnektorliste der Funktionspakete"}
    cp = re.compile(r'^\s+\d+\s+([a-zA-Z0-9_-]+)$') #find CPU name in file
    bg = re.compile(r'^\s+\d+\s+%s'%(bg_lang_options[SIMADYND_LANG])) # find beginning of link information
    en = re.compile(r'^\s*\d+\s+\*')
    ch = re.compile(r'^\s*\d+\s+FP-([a-zA-Z0-9_\-@]+)')
    lk = re.compile(r'^\s*\d+\s+(\$[a-zA-Z0-9_\-@]+)\s+([\w]+)\s+((<)|(>))\s+([\w]+)\s+([a-zA-Z0-9_-]+)$')
    nolk = re.compile(r'^\s*\d+\s+====\s+') #  62	   ==== Im FP sind keine FP-Verbindungen projektiert
    
    for i in sd_chart_list:
        for (k,v) in sd_chart_list[i].items():
            chart_lines = sd_open_chart(v)
            print("Parsing cpu file '%s'  "%(k))
            cpu_mark = False
            beg_mark = False
            end_mark = False
            for l in chart_lines:
                ##Find CPU name
                if not cpu_mark:
                    cp_s = cp.search(l)
                    if cp_s :
                        cpu = cp_s.group(1)
                        cpu_mark = True
                        print(cpu)   
                else:
                    if not beg_mark:
                        if bg.search(l):
                            beg_mark = True
                    else:
                        #check end
                        if en.search(l):
                            end_mark = True
                            break
                        else:
                            #main block:
                            ch_s = ch.search(l)
                            if ch.search(l): #if chart beginning 1739	FP-MONIW1
                                chart = ch_s.group(1)
                            else:
                                lk_s = lk.search(l)
                                if lk_s: #if chart link definition 1744	   $DMALB1 T5 < PN     PN-D01_P1
                                    #'cpu','chart','out','link','cpu_source'
                                    data_link_map_list.append([cpu, chart.upper(),lk_s.group(3),lk_s.group(1),lk_s.group(7)])
                                else:
                                    #Check if there is no FP connection in CPU -> add empty entry
                                    if nolk.search(l):
                                        data_link_map_list.append([cpu, chart.upper(),None,None,None])
    data_link_map_df = pd.DataFrame(data_link_map_list, columns=['cpu','chart','out','link','cpu_source'])
    return data_link_map_df

def sd_get_chart_cpu(data_link_map_df,chart):
    try:
        result = data_link_map_df[data_link_map_df['chart']==chart].iloc[0]['cpu']
    except IndexError:
        result = -1
    return result
def sd_get_block_link_source_cpu(data_link_map_df,link,chart):
    #print("debug:",link,chart)
    return data_link_map_df[(data_link_map_df['chart']==chart)&\
                            (data_link_map_df['link']==link)&\
                            (data_link_map_df['out']=="<")].iloc[0]['cpu_source']

def sd_get_link_partner(data_link_source_df,cpu_link):
    try:
        return data_link_source_df[(data_link_source_df['cpu_link']==cpu_link)].iloc[0]['chart_block_signal']
    except IndexError:
        return None
    
###########LIBRARY CELL###########
#find partner link and put in 'value_linked' column.
#This column should be used for comparison
def sd_rect_data_s1(data_df,data_link_source_df):
    def lookup(x):
        partner = sd_get_link_partner(data_link_source_df,x)
        return partner
    data_df['value_linked'] = data_df['cpu_link'].apply(lookup)
    return data_df

#copy values in 'value_linked' - to get proper view: value or link to other block. 
#This column should be used for comparison
#Also convert in-chart link to absolute representation e.g. ENMESS.Q => MONIW1/ENMESS.Q
def sd_rect_data_s2(data_df):
    def lookup(x):
        if x['value_linked'] is None:
            if re.search(r'^[a-zA-Z0-9_\-@]+\.[a-zA-Z0-9_\-@]+$',str(x['value'])) and not re.search(r'^[0-9]+\.[0-9]+$',str(x['value'])):
                if not x['type_base'] in ['CR','NK','TR','RR']:
                    return x['chart']+"\\"+x['value']
                else:
                    return x['value']
            else:
                #convert output 'link' array to string and return to 'value_linked'
                #TODO: doesn't work with $links -> should be extended for this case.
                if x['value'] is None and x['link']:
                    return ','.join(x['link'])
                else:
                    return x['value']  
        else:
            return x['value_linked']
    data_df['value_linked'] = data_df.apply(lookup,axis = 1)
    return data_df

#create df row with link source structure: chart_block_signal,cpu_link
def sd_get_source_link(block_data_list):
    #block_data_list: (NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE,CPU,CPU_SOURCE,CPU_LINK,CHART,BLOCK ),LINKS
    m = re.compile(r"^\$\w+")
    out_df = []
    for j in block_data_list:
        if j[INOUT]==">":
            if(j[INOUT]):
                val = str(j[VALUE])
                n = m.search(val) 
                if n:
                    #chart\block.signal,cpu_link
                    chart_block_signal = j[CHART].upper()+'\\'+j[BLOCK].upper()+'.'+j[NAME].upper()
                    cpu_link = j[CPU]+'_'+val.split()[0] #convert '$LINK PN' => '$LINK'
                    #print(chart_block_signal,"=>",cpu_link)
                    out_df.append([chart_block_signal,cpu_link])
            #look for source link in link array
            for k in j[LINKS]:
                if(k):
                    n = m.search(str(k)) 
                    if n:
                        #chart\block.signal,cpu_link
                        chart_block_signal = j[CHART].upper()+'\\'+j[BLOCK].upper()+'.'+j[NAME].upper()
                        cpu_link = j[CPU]+'_'+str(k).split()[0] #convert '$LINK PN' => '$LINK'
                        #print(chart_block_signal,"=>",cpu_link)
                        out_df.append([chart_block_signal,cpu_link])
    return out_df

#generate data of all charts and generate data of link sources
def sd_build_data_s1(sd_chart_list,data_link_map_df):
    data_list = []
    data_link_source_list = []
    for i in sd_chart_list:
        for (k,v) in sd_chart_list[i].items(): 
            chart = k.upper()
            chart_lines = sd_open_chart(v)
            chart_mapping = sd_get_chart_map(chart_lines)
            if chart_mapping==-1:
                print("Parsing '%s' chart with 0 functions"%(chart))
                continue
            else:
                print("Parsing '%s' chart with %s functions"%(chart,len(chart_mapping)))
            
            for block_map in chart_mapping:
                #generating array 'signal','type_base','inout','type_conv','value','link','chart','block','cpu'
                block_lines = chart_lines[chart_mapping[block_map][0]:chart_mapping[block_map][1]]
                #get block data: NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE,CPU,CPU_SOURCE,CPU_LINK,CHART,BLOCK
                block_data = sd_get_block_data(block_lines,chart,block_map,data_link_map_df)
                if block_data == -1:
                    print("Parsing cancelled: '%s' chart is not used in SimadynD program"%(chart))
                    break
                #print("before ",block_data)
                #print("-- ")
                block_data_list = sd_conv_block_data_to_list(block_data)
                #print("after ",block_data_list)
                #print("-- ")
                #Add block data to accumulation list
                data_list.extend(block_data_list)
                #create link source array
                block_link_source_list = sd_get_source_link(block_data_list)
                
                if len(block_link_source_list)!=0:
                    #print(block_link_source_list)
                    data_link_source_list.extend(block_link_source_list) 
    #Convert to DataFrame for easy work
    data_link_source_df = pd.DataFrame(data_link_source_list, columns=['chart_block_signal','cpu_link'])
    data_df = pd.DataFrame(data_list, columns=['signal','type_base','inout','type_conv','value','cpu',\
                                                         'cpu_source','cpu_link','chart','block','link'])
    data_df["block"] = data_df["block"].astype(str)
    return data_df,data_link_source_df


#get block data: signals, values, links, data types...
def sd_get_block_data(lines,chart,block,data_link_map_df):
    mark_signal_row_name = None
    mark_signal_row_data = list([[],[]]) #[[signal data],[links]]
    mark_block_row_data = []
    count = 0;
    #get CPU name
    cpu = sd_get_chart_cpu(data_link_map_df,chart)
    #If no cpu found, it means chart is not used in SD software:
    if cpu == -1:
        return -1
    for l in lines:
        line_check_result = sd_check_line_signal(l)
        if line_check_result:
            if line_check_result==1: 
                #if found a signal line
                count = count+1
                #add previous signal to preparation table
                if mark_signal_row_name:
                    #print("Old name:",mark_signal_row_name)
                    mark_block_row_data.append(mark_signal_row_data)
                    mark_signal_row_data = list([[],[]]) #reset buffer table
                #NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE
                mark_signal_row_data[0], extra_link = sd_get_line_data(l)
                mark_signal_row_data[0].append(cpu)
                #If value is $LINK here is a logic:
                #Copy value $LINK to link column
                if re.search(r'^\$',str(mark_signal_row_data[0][VALUE])) and mark_signal_row_data[0][INOUT]=="<":
                    link = mark_signal_row_data[0][VALUE].split()[0] #CONVERT '$LINK PN' => '$LINK'
                    mark_signal_row_data[1].append(link)
                    #get CPU_SOURCE
                    cpu_source = sd_get_block_link_source_cpu(data_link_map_df,link,chart)
                    mark_signal_row_data[0].append(cpu_source)
                    #get CPU_LINK   (means CPU_SOURCE_LINK)
                    mark_signal_row_data[0].append(cpu_source+"_"+link)
                else:
                    #get CPU_SOURCE
                    mark_signal_row_data[0].append(cpu)
                    #get CPU_LINK   (means CPU_SOURCE_LINK)
                    mark_signal_row_data[0].append(None) 
                #in case if we have double link like: $WT_EXT,!PDL1W1.0026
                if(extra_link):
                    #CONVERT '$LINK PN' => '$LINK'
                    mark_signal_row_data[1].append(extra_link.split()[0])
                mark_signal_row_data[1]
                mark_signal_row_name = mark_signal_row_data[0][NAME]
                #get CHART,BLOCK
                mark_signal_row_data[0].append(chart)
                mark_signal_row_data[0].append(block)
            elif line_check_result==2:
                #if found a link line: (SOMETHING,SOMETHING2)
                mark_signal_row_data[1].extend(sd_get_line_link(l).split(','))
            #print(mark_signal_row_data)            
        else:
            pass #skip string
    #Add last signal to preparation storage
    mark_block_row_data.append(mark_signal_row_data)
    return mark_block_row_data

#extract data from signal line
def sd_get_line_data(line): 
    m = re.compile(r"""
        ^\s*\d+\s+ #string number
        (\w+)\s+   #signal name 
        (\w+)\s+   #signal type 
        ([<>=-])\s*  #signal delimiter 
        (@TYP=(..)[\s]*,?[\s]*)? #singal type
        (   #value
        ([a-zA-Z0-9_.]+)|  #can be connection to other block
        ([$%!\[\]/()a-zA-Z0-9\s_.*#\-\+]+)|  #can be value
        ([$a-zA-Z0-9_]+\s*[$a-zA-Z0-9_]*)|  #can be virtual connection
        ([#0-9.]+\s*\[.*\])|  #can be time with spaces
        ([0-9B]+\s+[0-9]+)|  #can be hex in bit representation
        ('.*')| #can be text
        (\*[a-zA-Z0-9_.]+)  #can hardware connection: *ADRCO1
        )? #end of value
        ([\s]*,)? #divider
        (\![a-zA-Z0-9_\.]+)?
        ([\s]*,)?
        ('.*')? #link comment
        (,.*)? #some comment
        (\s*".*")? #signal comment
        (&)? #check next string
        (\s)*
        ($) #end of string
        """, re.X)
    #print(line)
    n = m.search(line)
    extra_link = None
    try:
        #Check if value is XXXX but behind is telegram connection like !XXX.YY
        
        if n.group(15):
            check_val = n.group(15)
            if n.group(6):
                if re.search("\$.+",n.group(6)):
                    extra_link = n.group(6)
        else:
            check_val = n.group(6)
        out = list([n.group(1),n.group(2),n.group(3),n.group(5),check_val]), extra_link
    except:
        #print line for debugging
        print(line) 
        raise   
    #     #name      type       dir        type_conv  value 
    #list([n.group(1),n.group(2),n.group(3),n.group(5),n.group(6)]), extra_link
    return out

In [9]:
###########EXECUTION CELL###########
if(COMPARISON_TYPE==COMTYPE_SDSD):
    sd1_chart_list = sd_get_chart_list(PATH_TO_STRUCG_1,SD_RACK_NAMES) 
    sd1_cpu_charts_list = sd_get_cpu_charts_list(PATH_TO_STRUCG_1,SD_RACK_NAMES)
    sd2_chart_list = sd_get_chart_list(PATH_TO_STRUCG_2,SD_RACK_NAMES) 
    sd2_cpu_charts_list = sd_get_cpu_charts_list(PATH_TO_STRUCG_2,SD_RACK_NAMES) 
elif  (COMPARISON_TYPE==COMTYPE_SDTDC):  
    sd1_chart_list = sd_get_chart_list(PATH_TO_STRUCG_1,SD_RACK_NAMES) 
    sd1_cpu_charts_list = sd_get_cpu_charts_list(PATH_TO_STRUCG_1,SD_RACK_NAMES)
    tdc2_data_a1 = parse_tdc_source_file(PATH_TO_DATA,PATH_TO_TDC_DATA_2)
elif  (COMPARISON_TYPE==COMTYPE_TDCSD):  
    raise Exception('TDC to SD not implemented yet')
    tdc1_data_a1 = parse_tdc_source_file(PATH_TO_DATA,PATH_TO_TDC_DATA_1)
    sd2_chart_list = sd_get_chart_list(PATH_TO_STRUCG_2,SD_RACK_NAMES) 
    sd2_cpu_charts_list = sd_get_cpu_charts_list(PATH_TO_STRUCG_2,SD_RACK_NAMES)
elif  (COMPARISON_TYPE==COMTYPE_TDCTDC):  
    raise Exception('TDC to TDC not implemented yet')
    tdc1_data_a1 = parse_tdc_source_file(PATH_TO_DATA,PATH_TO_TDC_DATA_1)
    tdc2_data_a1 = parse_tdc_source_file(PATH_TO_DATA,PATH_TO_TDC_DATA_2)

In [55]:
###########TEST CELL, IGNORE###########
#n = re.search("^(((?!old).)*).((cfp)|(ofp))$", 'pray1___20131208.cfp')
test_txt = []
test_txt.append(" 129	X01 V2 < 0H0,!E2C113.0011&")
test_txt.append(" 325	I1  B1 < VIHW09.Q15&")
test_txt.append("1397	CCI N2 < DMVI04.Y ")
test_txt.append("1447	I3  B1 < VII250.Q9  ,'ENNROLRD' \"SECTION NOT READY FOR ROLLING\"")
test_txt.append("1229	AD  NK = *ADRCO1,'DO5I2X6A'")
test_txt.append(" 100	IS  V2 < $VST1LS\"Limit switches stand movements 1\"")
test_txt.append('  81	IS  V2 < $VISW17')
test_txt.append(" 163	Q1  B1 > ,'Man__CPS'            \"MAINTENANCE COIL PREP STATION\"")
test_txt.append(' 115	I1  B1 < VIHW17.Q2&')
test_txt.append(' 519	X1  N2 < @TYP=V2,0HFFFF')
test_txt.append(" 150	Y   N2 > @TYP=O2,'RP_ALL'&")
test_txt.append(' 520	X2  N2 < @TYP=V2,LB2.QS&')
test_txt.append(' 527	QS  V2 > $VGAU02                "Diagnose 2nd direction"')
test_txt.append(' 528	Q   B1 > ')
test_txt.append("  31	AD  NK = D02_I1.X8C")
test_txt.append("  32	DM  B1 - 1")
test_txt.append("  22	AR  NS - 'B1G1X1'")
test_txt.append("  21	CTS CR - D0300A")
test_txt.append("1151	T1  NS - '@F@85@ EN QFC Coil P. Shear Tab. Air Float'")
test_txt.append("1199	X1  N2 < 0%")
test_txt.append("1200	X2  N2 < SEND07.QS")
test_txt.append("1201	I   B1 < EN_MES.Q,'EN_MES'&")
test_txt.append(" 566	LZU NF < #10000[ms],'RunTime '  \"Run time monitoring\"")
test_txt.append("1207	X2  N2 < 18.298%                \"18,3 representa 3000 kg\"")
test_txt.append("1283	X2  NF < #0")
test_txt.append("1304	X2  NF < #26,'GRAUS'")
test_txt.append(" 905	X2  NF < #100.0000000E3")
test_txt.append("1074	Y   NF > @TYP=TF,$PTTI_A,'PTTI_AUT'&")
test_txt.append("1073	X   NF < @TYP=TF,10[S]")
test_txt.append("2983	X2  NF < #530.0000000E-3,'m'")
test_txt.append(" 439	FBU NF < #500[ms],'Feedback'    \"Feedback monitoring\"")
test_txt.append(" 494	I   B1 < STG641.Q")
test_txt.append(" 495	Y   N2 > @TYP=V2")
test_txt.append("  17	X1  NF < $XIHW05,SCAL=1[m]")
test_txt.append(" 374	T   TF <  3           [s]")
test_txt.append(" 169	T   TF <            2[s ]")
test_txt.append("2946	OR  V2 < 0B00000000 00000000")
test_txt.append(" 108	X05 NF < #2500.0[V/(m/s)],'ky'  \"Scaling factor for control output\"&")
test_txt.append("2756	CRT TR = !EXM504")
test_txt.append(" 337	Q1  B1 >")
test_txt.append("1407	T   TF < #2.5 [s]")
test_txt.append("  87	Q3  B1 > $E_STOP PN,'E_STOP'&")
test_txt.append(" 267	X   NF < #1E+6")
test_txt.append(' 602	X2  NF < #-0.200E-09[m/(N*s)]   "Umrechnung [N] auf [m]"')
test_txt.append(' 108	TD  TF < #1E-3          [S]      "Verzoegerungszeit"')
test_txt.append(' 541	Y07 NF > $WT_EXT,!PDL1W1.0026&')
test_txt.append('1590	Y   N2 > @TYP=I2 ,$DASTID PN    "Band ID for MMI(Example: Band  Nr.17 "')
test_txt2 = " 439	FBU NF < #500[ms],'Feedback'    \"Feedback monitoring\""


#m = re.compile(r"^\s*\d+\s+(\w+)\s+(\w+)\s+([<>])\s+(@TYP=(..),)?([^@\"',&]+)([,\"'&]|$)", )
m = re.compile(r"""
        ^\s*\d+\s+ #string number
        (\w+)\s+   #signal name 
        (\w+)\s+   #signal type 
        ([<>=-])\s*  #signal delimiter 
        (@TYP=(..)[\s]*,?[\s]*)? #singal type
        (   #value
        ([a-zA-Z0-9_.]+)|  #can be connection to other block
        ([$%!\[\]/()a-zA-Z0-9\s_.*#\-\+]+)|  #can be value
        ([$a-zA-Z0-9_]+\s*[$a-zA-Z0-9_]*)|  #can be virtual connection
        ([#0-9.]+\s*\[.*\])|  #can be time with spaces
        ([0-9B]+\s+[0-9]+)|  #can be hex in bit representation
        ('.*')| #can be text
        (\*[a-zA-Z0-9_.]+)  #can hardware connection: *ADRCO1
        )? #end of value
        ([\s]*,)? #divider
        (\![a-zA-Z0-9_\.]+)?
        ([\s]*,)?
        ('.*')? #link comment
        (,.*)? #some comment
        (\s*".*")? #signal comment
        (&)? #check next string
        (\s)*
        ($) #end of string
        """, re.X)
k = re.compile(r"^\s*\d+\s+(\w+)\s+(\w+)\s+([<>=-])")

for a in test_txt:
    n = m.search(a)
    print(a)
    #print(n.group(1),n.group(2),n.group(3),n.group(5),n.group(6),n.group(13))
    #print(n)
    print(n.group(6),n.group(15),"\n")
    #print(n.group(5),"\n")
    #print(n)
    
#n = m.search(test_txt2)
#print("OUT:",n.group(1),n.group(2),n.group(3),n.group(6),n.group(13))

#for a in test_txt:
#    n = k.search(a)
#    print(n)

#for a in test_txt:
#    n = k.search(a)
#    print(n)

 129	X01 V2 < 0H0,!E2C113.0011&
0H0 !E2C113.0011 

 325	I1  B1 < VIHW09.Q15&
VIHW09.Q15 None 

1397	CCI N2 < DMVI04.Y 
DMVI04.Y None 

1447	I3  B1 < VII250.Q9  ,'ENNROLRD' "SECTION NOT READY FOR ROLLING"
VII250.Q9 None 

1229	AD  NK = *ADRCO1,'DO5I2X6A'
*ADRCO1 None 

 100	IS  V2 < $VST1LS"Limit switches stand movements 1"
$VST1LS None 

  81	IS  V2 < $VISW17
$VISW17 None 

 163	Q1  B1 > ,'Man__CPS'            "MAINTENANCE COIL PREP STATION"
None None 

 115	I1  B1 < VIHW17.Q2&
VIHW17.Q2 None 

 519	X1  N2 < @TYP=V2,0HFFFF
0HFFFF None 

 150	Y   N2 > @TYP=O2,'RP_ALL'&
'RP_ALL' None 

 520	X2  N2 < @TYP=V2,LB2.QS&
LB2.QS None 

 527	QS  V2 > $VGAU02                "Diagnose 2nd direction"
$VGAU02                 None 

 528	Q   B1 > 
None None 

  31	AD  NK = D02_I1.X8C
D02_I1.X8C None 

  32	DM  B1 - 1
1 None 

  22	AR  NS - 'B1G1X1'
'B1G1X1' None 

  21	CTS CR - D0300A
D0300A None 

1151	T1  NS - '@F@85@ EN QFC Coil P. Shear Tab. Air Float'
'@F@85@ EN QFC Coil P. Shear Tab. Air Float'

In [46]:
###########TEST CELL, IGNORE###########
#test_txt="   &	V4.2.6 FP-COP CHK 22.02.;9 07:54  (MP-TX04.PN-D05_P2 13.09.;6 05:02)"
#test_txt = "   1 PN-D01_P1"
#test_txt = "1832	*"
#test_txt = ""
#test_txt = "  89 Connector list of the function packets"
test_txt = " 402	   $WWSTAT T5 <  FL    PN-D01_P1"
#test_txt = " 830	   $WWSTAT T4 <  PN     PN-D01_P1"
#test_txt = "1816	FP-@SEND1,COM"
#test_txt = "1679	   $VINS02 T4 < PN     PN-D01_P1"
#test_txt = " 830	   $DIADE  T4 < PN     PN-D01_P1"
#test_txt = "ENMESS.Q1231"

#cp = re.compile(r'^\s+\d+\s+([a-zA-Z0-9_-]+)$')
#cp = re.compile(r'^\s+\d+\s+Connector list of the function packets')
#cp = re.compile(r'^\s*\d+\s+\*')
#cp = re.compile(r'^\s*\d+\s+FP-([a-zA-Z0-9_\-@]+)')
cp = re.compile(r'^\s*\d+\s+(\$[a-zA-Z0-9_\-@]+)\s+([\w]+)\s+((<)|(>))\s+([\w]+)\s+([a-zA-Z0-9_-]+)$')
#cp = re.compile(r'^[a-zA-Z0-9_\-@]+\.[a-zA-Z0-9_\-@]+$')
cp_s = cp.search(test_txt)
print(cp_s)
#print(cp_s.group(1),cp_s.group(3),cp_s.group(7))

<_sre.SRE_Match object; span=(0, 37), match=' 402\t   $WWSTAT T5 <  FL    PN-D01_P1'>


In [10]:
###########EXECUTION CELL###########
if(COMPARISON_TYPE==COMTYPE_SDSD):
    sd1_data_lm = sd_build_cpu_map(sd1_cpu_charts_list)
    sd1_data_s1,sd1_data_ls1 = sd_build_data_s1(sd1_chart_list,sd1_data_lm)
    sd2_data_lm = sd_build_cpu_map(sd2_cpu_charts_list)
    sd2_data_s1,sd2_data_ls1 = sd_build_data_s1(sd2_chart_list,sd2_data_lm)
elif  (COMPARISON_TYPE==COMTYPE_SDTDC):  
    sd1_data_lm = sd_build_cpu_map(sd1_cpu_charts_list)
    sd1_data_s1,sd1_data_ls1 = sd_build_data_s1(sd1_chart_list,sd1_data_lm)
elif  (COMPARISON_TYPE==COMTYPE_TDCSD):  
    raise Exception('TDC to SD not implemented yet')
    sd2_data_lm = sd_build_cpu_map(sd2_cpu_charts_list)
    sd2_data_s1,sd2_data_ls1 = sd_build_data_s1(sd2_chart_list,sd2_data_lm)
elif  (COMPARISON_TYPE==COMTYPE_TDCTDC):  
    raise Exception('TDC to TDC not implemented yet')


Parsing cpu file '1sx01'  
PN-D01_P1
Parsing cpu file '2sx01'  
PN-D04_P2
Parsing cpu file '3sx01'  
PN-D05_P3
Parsing cpu file '4sx01'  
PN-D06_P4
Parsing cpu file '5sx01'  
PN-D08_P5
Parsing '@RCV_1' chart with 1 functions
Parsing '@SND_1' chart with 14 functions
Parsing '@SND_2' chart with 2 functions
Parsing '@SND_3' chart with 2 functions
Parsing '@SND_4' chart with 2 functions
Parsing '@SND_5' chart with 2 functions
Parsing 'BLOC_1' chart with 55 functions
Parsing 'BLOC_5' chart with 53 functions
Parsing 'COP1_2' chart with 35 functions
Parsing 'COPY_1' chart with 3 functions
Parsing 'COW1_2' chart with 57 functions
Parsing 'CTRC_1' chart with 202 functions
Parsing 'CTRC_2' chart with 126 functions
Parsing 'CTRC_3' chart with 17 functions
Parsing 'CTRC_4' chart with 43 functions
Parsing 'CTRC_5' chart with 87 functions
Parsing 'CTRH_1' chart with 19 functions
Parsing 'CTRS_1' chart with 13 functions
Parsing 'CTRS_2' chart with 8 functions
Parsing 'CTRS_3' chart with 8 functions
P

In [29]:
sd1_data_s1

,signal,type_base,inout,type_conv,value,cpu,cpu_source,cpu_link,chart,block,link
0,X1,N2,<,None,D324.Y,PN-D01_P1,PN-D01_P1,None,DAT__1,D324,[]
1,X2,N2,<,I2,$DBMCN3,PN-D01_P1,PN-D01_P1,None,DAT__1,D324,[]
2,I,B1,<,None,D310.Q,PN-D01_P1,PN-D01_P1,None,DAT__1,D324,[]
3,Y,N2,>,None,None,PN-D01_P1,PN-D01_P1,None,DAT__1,D324,"[D623.X1, D324.X1, D423.X1, D523.X1]"
4,X01,N2,<,I2,0,PN-D01_P1,PN-D01_P1,None,MMID_1,II2001,[]
5,X02,N2,<,I2,0,PN-D01_P1,PN-D01_P1,None,MMID_1,II2001,[]
6,X03,N2,<,I2,0,PN-D01_P1,PN-D01_P1,None,MMID_1,II2001,[]
7,X04,N2,<,I2,0,PN-D01_P1,PN-D01_P1,None,MMID_1,II2001,[]
8,X05,N2,<,I2,0,PN-D01_P1,PN-D01_P1,None,MMID_1,II2001,[]
9,X06,N2,<,I2,0,PN-D01_P1,PN-D01_P1,None,MMID_1,II2001,[]


In [22]:
sd1_data_lm[sd1_data_lm["link"]=="$DBMCN3"]


,cpu,chart,out,link,cpu_source
75,PN-D01_P1,MMID_1,>,$DBMCN3,PN-D01_P1
551,PN-D01_P1,DAT__1,<,$DBMCN3,PN-D01_P1
1182,PN-D03_P2,MMID_2,>,$DBMCN3,PN-D03_P2


In [9]:
###########EXECUTION CELL###########
#save intermidiate result
if(COMPARISON_TYPE==COMTYPE_SDSD):
    sd1_data_s1.to_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP1))
    sd1_data_ls1.to_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP2))
    sd1_data_lm.to_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP3))
    sd2_data_s1.to_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP1))
    sd2_data_ls1.to_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP2))
    sd2_data_lm.to_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP3))
elif  (COMPARISON_TYPE==COMTYPE_SDTDC):  
    sd1_data_s1.to_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP1))
    sd1_data_ls1.to_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP2))
    sd1_data_lm.to_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP3))
    tdc2_data_a1.to_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP1))
elif  (COMPARISON_TYPE==COMTYPE_TDCSD):  
    raise Exception('TDC to SD not implemented yet')
    tdc1_data_a1.to_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP1))
    sd2_data_s1.to_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP1))
    sd2_data_ls1.to_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP2))
    sd2_data_lm.to_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP3))
elif  (COMPARISON_TYPE==COMTYPE_TDCTDC):  
    raise Exception('TDC to TDC not implemented yet')
    tdc1_data_a1.to_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP1))
    tdc2_data_a1.to_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP1))

In [23]:
###########EXECUTION CELL###########
#restore intermediate result
if(COMPARISON_TYPE==COMTYPE_SDSD):
    sd1_data_s1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP1),index_col=0)
    sd1_data_ls1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP2),index_col=0)
    sd1_data_lm = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP3),index_col=0)
    sd2_data_s1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP1),index_col=0)
    sd2_data_ls1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP2),index_col=0)
    sd2_data_lm = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP3),index_col=0)
elif  (COMPARISON_TYPE==COMTYPE_SDTDC):  
    sd1_data_s1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP1),index_col=0)
    sd1_data_ls1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP2),index_col=0)
    sd1_data_lm = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP3),index_col=0)
    tdc2_data_a1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP1),index_col=0)
elif  (COMPARISON_TYPE==COMTYPE_TDCSD):  
    raise Exception('TDC to SD not implemented yet')
    tdc1_data_a1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP1),index_col=0)
    sd2_data_s1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP1),index_col=0)
    sd2_data_ls1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP2),index_col=0)
    sd2_data_lm = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP3),index_col=0)
elif  (COMPARISON_TYPE==COMTYPE_TDCTDC):  
    raise Exception('TDC to TDC not implemented yet')
    tdc1_data_a1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP1),index_col=0)
    tdc2_data_a1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP1),index_col=0)

In [24]:
###########EXECUTION CELL###########
#find links for sd
if(COMPARISON_TYPE==COMTYPE_SDSD):
    sd1_data_s2 = sd_rect_data_s1(sd1_data_s1.copy(),sd1_data_ls1)
    sd1_data_s3 = sd_rect_data_s2(sd1_data_s2.copy())
    sd2_data_s2 = sd_rect_data_s1(sd2_data_s1.copy(),sd2_data_ls1)
    sd2_data_s3 = sd_rect_data_s2(sd2_data_s2.copy())
elif  (COMPARISON_TYPE==COMTYPE_SDTDC):  
    sd1_data_s2 = sd_rect_data_s1(sd1_data_s1.copy(),sd1_data_ls1)
    sd1_data_s3 = sd_rect_data_s2(sd1_data_s2.copy())
elif  (COMPARISON_TYPE==COMTYPE_TDCSD):  
    raise Exception('TDC to SD not implemented yet')
    sd2_data_s2 = sd_rect_data_s1(sd2_data_s1.copy(),sd2_data_ls1)
    sd2_data_s3 = sd_rect_data_s2(sd2_data_s2.copy())
elif  (COMPARISON_TYPE==COMTYPE_TDCTDC):  
    raise Exception('TDC to TDC not implemented yet')

In [20]:
sd1_data_s2

,signal,type_base,inout,type_conv,value,cpu,cpu_source,cpu_link,chart,block,link,value_linked
0,X1,N2,<,None,D324.Y,PN-D01_P1,PN-D01_P1,None,DAT__1,D324,[],None
1,X2,N2,<,I2,$DBMCN3,PN-D01_P1,PN-D01_P1,None,DAT__1,D324,[],None
2,I,B1,<,None,D310.Q,PN-D01_P1,PN-D01_P1,None,DAT__1,D324,[],None
3,Y,N2,>,None,None,PN-D01_P1,PN-D01_P1,None,DAT__1,D324,"[D623.X1, D324.X1, D423.X1, D523.X1]",None
4,X01,N2,<,I2,0,PN-D01_P1,PN-D01_P1,None,MMID_1,II2001,[],None
5,X02,N2,<,I2,0,PN-D01_P1,PN-D01_P1,None,MMID_1,II2001,[],None
6,X03,N2,<,I2,0,PN-D01_P1,PN-D01_P1,None,MMID_1,II2001,[],None
7,X04,N2,<,I2,0,PN-D01_P1,PN-D01_P1,None,MMID_1,II2001,[],None
8,X05,N2,<,I2,0,PN-D01_P1,PN-D01_P1,None,MMID_1,II2001,[],None
9,X06,N2,<,I2,0,PN-D01_P1,PN-D01_P1,None,MMID_1,II2001,[],None


In [13]:
###########LIBRARY CELL###########
#report generator of comparison of block in all SD1 and SD2 charts
def report_find_miss_blocks_sdsd (sd1_data_sd,sd2_data_sd, to_file = True, timestamp = datetime.datetime.now()):
    README_HEADER = COMPARATOR_README +"\
\r\nReport: find missing blocks in sources\
\r\nDate ant time of report generation: %s \
\r\n===================================================================================\
\r\n"%(str(timestamp))
    
    def generate_report(f):
        print(README_HEADER,file = f) 
        sd1_chartblock_sel = sd1_data_sd["chart"]+"/"+sd1_data_sd["block"]
        sd1_uniq_blocks = sd1_chartblock_sel.unique()
        #tdc_uniq_blocks = data_tdc['chartblock'].unique()
        sd2_chartblock_sel = sd2_data_sd["chart"]+"/"+sd2_data_sd["block"]
        sd2_uniq_blocks = sd2_chartblock_sel.unique()
        #look for A in B
        for i in sd1_uniq_blocks:
            if not i in sd2_uniq_blocks:
                print("Missing block in SD#2: ",i,"\r\n",file = f) 
        #look for B in A
        for i in sd2_uniq_blocks:
            if not i in sd1_uniq_blocks:
                print("Missing block in SD#1: ",i,"\r\n",file = f) 
  
    if to_file:
        file_suffix = "@_compare_blocks_sdsd_%s"%(str(timestamp.date()))
        file_prefix = "report_@"
        try:
            # Create target Directory
            os.mkdir(os.path.join(PATH_TO_REPORTS))
        except FileExistsError:
            pass
        with open(os.path.join(PATH_TO_REPORTS,(file_prefix+SD_RACK_NAMES[0]+file_suffix+".txt")), 'w') as f: 
            generate_report(f)
    else:
        generate_report(sys.stdout)
        
#report generator of comparison of block in all SD and TDC charts
def report_find_miss_blocks_sdtdc (data_sd,data_tdc, to_file = True, timestamp = datetime.datetime.now()):
    README_HEADER = COMPARATOR_README +"\
\r\nReport: find missing blocks in sources\
\r\nDate ant time of report generation: %s \
\r\n===================================================================================\
\r\n"%(str(timestamp))
    
    def generate_report(f):
        print(README_HEADER,file = f) 
        chartblock_sel = data_sd["chart"]+"/"+data_sd["block"]
        sd_uniq_blocks = chartblock_sel.unique()
        tdc_uniq_blocks = data_tdc['chartblock'].unique()
                
        #look for A in B
        for i in sd_uniq_blocks:
            if not i in tdc_uniq_blocks:
                print("Missing block in TDC: ",i,"\r\n",file = f) 
        #look for B in A
        for i in tdc_uniq_blocks :
            if not i in sd_uniq_blocks:
                print("Missing block in SD: ",i,"\r\n",file = f)   
                
    if to_file:
        file_suffix = "@_compare_blocks_sdtdc_%s"%(str(timestamp.date()))
        file_prefix = "report_@"
        try:
            # Create target Directory
            os.mkdir(os.path.join(PATH_TO_REPORTS))
        except FileExistsError:
            pass
        with open(os.path.join(PATH_TO_REPORTS,(file_prefix+SD_RACK_NAMES[0]+file_suffix+".txt")), 'w') as f: 
            generate_report(f)
    else:
        generate_report(sys.stdout)
        
###########LIBRARY CELL###########
#####report of comparison of all SD blocks inputs, including links

        #['BOOL' 'INT' 'WORD' 'SDTIME' 'GLOBAL' 'STRING' 'REAL' 'DINT' 'BYTE' 'DWORD']
        #SD types:
        #'B1' - BOOL (0/1/link)
        #'TF' - Time (ffff.fff[s/min/S/ms])
        #'N2' - Integer (ddddd)
        #'V2' - Integer HEX
        #'MR' - Message link (e.g MESY_1)(Inout = "-")
        #'NS' - String (Inout = "-")
        #'O2' - Integer (dddddd)
        #'CR' - Hardware connectors, string
        #'V4' - Dword HEX
        #'NF' - Floating point
        #'V1' - Byte HEX   
        #'I2' - Integer
        #'NK' - Hardware connectors, string (Inout = "=")
        #'TR' - Telegram transmit connector e.g. !EXM504 (Inout = "=")
        #'N4' - Integer 32 bits (%/HEX/ddd)
        #'RR' - Telegram recieve connector e.g. !T4EX05 (Inout = "=") 
        


def report_comp_signals_sdsd (s1_inp_data,s2_inp_data, to_file = True, timestamp = datetime.datetime.now()):
    README_HEADER = COMPARATOR_README +"\
\r\nReport: compare all SD#1 inputs (including links) with SD#2\
\r\nDate ant time of report generation: %s \
\r\n===================================================================================\
\r\n"%(str(timestamp))
    sd1_data_sd = s1_inp_data.copy()
    sd2_data_sd = s2_inp_data.copy()
    
    def clean_sd_tdc_str_values(s1_value,s2_value,sd_type,sd_type_conv,chart_block_signal,source_dest_text,f):
        #check if values are empty
        if  pd.isnull(s1_value):
            if pd.isnull(s2_value):
                return True
            else:
                result = False;
        else:
            if pd.isnull(s2_value):
                result = False;
            else:
                result = s1_value == s2_value;
            
        if not result:
            print("\r\nDifference found in ",source_dest_text,chart_block_signal,": ",s1_value,'==>>> ',s2_value,file = f)
        return result

    def generate_report(f):
        print(README_HEADER,file = f) 
        
        #filter sd data frame
        sd1_data_sd_filtered = sd1_data_sd[(sd1_data_sd['type_base'].isin(filt_type_base))&(sd1_data_sd['inout'].isin(filt_inout))]
        sd2_data_sd_filtered = sd2_data_sd[(sd2_data_sd['type_base'].isin(filt_type_base))&(sd2_data_sd['inout'].isin(filt_inout))]
        
        sd1_data_sd_filtered['chart_block_signal'] = sd1_data_sd_filtered["chart"]+"/"+sd1_data_sd_filtered["block"]+"."+sd1_data_sd_filtered["signal"]
        sd2_data_sd_filtered['chart_block_signal'] = sd2_data_sd_filtered["chart"]+"/"+sd2_data_sd_filtered["block"]+"."+sd2_data_sd_filtered["signal"]
        
        #connect links $XXXXX->$XXXXX
        d1 = dict(zip(sd2_data_sd_filtered['chart_block_signal'].values, sd2_data_sd_filtered['value_linked'].values))
        d2 = dict(zip(sd1_data_sd_filtered['chart_block_signal'].values, sd1_data_sd_filtered['value_linked'].values))
        
        sd1_data_sd_filtered['val_to_compare'] = sd1_data_sd_filtered['chart_block_signal'].map(d1)
        sd1_data_sd_filtered['compare_val_sd'] = sd1_data_sd_filtered.apply(lambda row: clean_sd_tdc_str_values(\
           row['value_linked'], row['val_to_compare'],row['type_base'], row['type_conv'],row['chart_block_signal'],"SD#1->SD#2 value:",f), axis=1)
        
        sd2_data_sd_filtered['val_to_compare'] = sd2_data_sd_filtered['chart_block_signal'].map(d2)
        sd2_data_sd_filtered['compare_val_sd'] = sd2_data_sd_filtered.apply(lambda row: clean_sd_tdc_str_values(\
           row['value_linked'], row['val_to_compare'],row['type_base'], row['type_conv'],row['chart_block_signal'],"SD#2->SD#1 value:",f), axis=1)
        
    if to_file:
        file_suffix = "@_compare_singals_sdsd_%s"%(str(timestamp.date()))
        file_prefix = "report_@"
        try:
            # Create target Directory
            os.mkdir(os.path.join(PATH_TO_REPORTS))
        except FileExistsError:
            pass
        with open(os.path.join(PATH_TO_REPORTS,(file_prefix+SD_RACK_NAMES[0]+file_suffix+".txt")), 'w') as f: 
            generate_report(f)
    else:
        generate_report(sys.stdout)
        
def report_comp_signals_sdtdc (inp_data_sd,inp_data_tdc, to_file = True, timestamp = datetime.datetime.now()):
    README_HEADER = COMPARATOR_README +"\
\r\nReport: compare SD signals (including links) with TDC \
\r\nDate ant time of report generation: %s \
\r\n===================================================================================\
\r\n"%(str(timestamp))
    data_sd = inp_data_sd.copy()
    data_tdc = inp_data_tdc.copy()
    def merge_tdc_link_value(l_value,l_link):
        if pd.isnan(l_link):
            return l_link
        else:
            return l_value
    
    def clean_sd_tdc_str_values(s_value,a_value,sd_type,sd_type_conv,chart_block_signal,inout,f):
        #modify HEX function
        def twos_complement(hexstr,bits,base=16):
            value = int(hexstr,base)
            if value & (1 << (bits-1)):
                value -= 1 << bits
            return value
 
        #for debugging
        #print("Working on:",s_value,a_value,sd_type,sd_type_conv)
        
        #check if values are empty
        if  pd.isnull(s_value):
            if pd.isnull(a_value):
                return True
            else:
                return False
        else:
            if pd.isnull(a_value):
                return False
        #regex for chart and link $ 
        con_check = re.compile(r'^[a-zA-Z0-9_\-@]+\\[a-zA-Z0-9_\-@]+\.[a-zA-Z0-9_\-@]+$')
        link_check = re.compile(r'^\$')
        link_check2 = re.compile(r'^\!')
        #check if SD is link
        #print("s_value:",s_value)
        #print("a_value:",a_value)
        if con_check.search(s_value) or link_check.search(s_value) or link_check2.search(s_value) \
        or ("'" in s_value):
            #print("Signal value of simadyn has chart connection")
            a_value_res=a_value
            s_value_res=s_value
        else:
            if con_check.search(a_value) or link_check.search(a_value) or link_check2.search(a_value):
                #SD is value but TDC is a link
                a_value_res=a_value
                s_value_res=s_value
            else:
                #SD and TDC are normal values
                if inout == '>':
                    #if its output and not link so we don't care
                    a_value_res,s_value_res=0,0 #any value
                elif sd_type=='NF':
                    a_value_res = float(a_value.replace(',','.'))
                    if '%' in s_value.split("[")[0]:
                        s_value_res = float(re.sub('[^0-9.E-]','', s_value.split("[")[0]))*0.01
                    else:
                        if sd_type_conv == 'TF':
                            units = 1000
                            un_txt = s_value.lower()
                            un_txt = un_txt.split("[")[1]
                            un_txt = un_txt.split("]")[0]
                            if un_txt=="s":
                                units = 1000
                            elif un_txt=="min":
                                units = 60000
                            elif un_txt=="ms":
                                units = 1
                            tmp_txt = re.sub('[^0-9.E-]','', s_value)
                            s_value_res = float(tmp_txt)*units 
                            #previous: s_value_res = int(math.floor(float(tmp_txt)*units)) - less accuracy
                        else: 
                            if "[" in s_value:
                                s_value_res = float(re.sub('[^0-9.E-]','', s_value.split("[")[0]))
                            else:
                                s_value_res = float(re.sub('[^0-9.E-]','', s_value))  
                elif sd_type=='B1':
                    if re.match(r'[10]$', s_value.strip()):
                        s_value_res = bool(int(s_value))
                    else:
                        print(s_value, a_value)
                        raise
                    a_value_res = bool(int(a_value))
                elif sd_type=='TF': 
                    a_value_res = float(a_value.strip('ms'))
                    units = 1000
                    un_txt = s_value.lower()
                    un_txt = un_txt.split("[")[1]
                    un_txt = un_txt.split("]")[0]
                    if un_txt=="s":
                        units = 1000
                    elif un_txt=="min":
                        units = 60000
                    elif un_txt=="ms":
                        units = 1
                    tmp_txt = re.sub('[^0-9.E-]','', s_value)
                    s_value_res = float(tmp_txt)*units #
                    #previous: s_value_res = int(math.floor(float(tmp_txt)*units)) - less accuracy

                elif sd_type in ['N2','N4','V2','O2','V4','V1','I2']:
                    #edit TDC value
                    if "16#" in a_value:
                        tmp_txt = a_value.split('16#')[1]
                        a_value_res = twos_complement(tmp_txt.strip(),16)    
                    else:
                        a_value_res = int(a_value)
                    #prepare SD value    
                    if "H" in s_value:
                        tmp_txt = s_value.split('0H')[1]
                        s_value_res = twos_complement(tmp_txt.strip(),16)
                    elif '%' in s_value:
                        s_value_res = int(round(float(re.sub('[^0-9.E-]','', s_value))*16384*0.01))
                    elif 'GRAD' in s_value:
                        s_value_res = int(round(float(re.sub('[^0-9.E-]','', s_value))*16384*0.001))
                    elif "B" in s_value:
                        tmp_txt = s_value.split('0B')[1]
                        tmp_txt =  tmp_txt.replace(" ", "")
                        s_value_res = twos_complement(tmp_txt.strip(),16,base = 2)
                    #special case for [GRAD] units. Eg 70.01 [GRAD]
                    elif "grad" in s_value.lower():
                        s_value_res = int(float(re.sub('[^0-9.E-]','', s_value))) 
                    else:
                        try:
                            s_value_res = int(re.sub('[^0-9.E-]','', s_value)) 
                        except ValueError:
                            print("ValueError: s_value=%s,a_value=%s,sd_type=%s,sd_type_conv=%s,chart_block_signal=%s"%(s_value,a_value,sd_type,sd_type_conv,chart_block_signal))
                            raise
                elif sd_type in ['TR','RR','NS','MR','CR']:
                    a_value_res = a_value.strip("'")
                    s_value_res = s_value.strip("'")
                elif sd_type in ['NK']:
                    a_value_res = a_value.strip("*")
                    s_value_res = s_value.strip("*")
                else:
                    print("\r\nFault in:",s_value,a_value,sd_type,sd_type_conv)
                    raise
        if not isinstance(s_value_res, str):
            if float(a_value_res)!=0 and float(s_value_res)!=0:
                result = abs((float(s_value_res)-float(a_value_res))/float(s_value_res))<0.0015
            else:
                result = s_value_res==a_value_res
        else:
            result = s_value_res==a_value_res
        if not result:
            print("\r\nDifference found in: ",chart_block_signal,"SD/TDC value:",s_value_res,'==>>> ',a_value_res,file = f)
        return result

    def generate_report(f):
        print(README_HEADER,file = f) 
        #this comparator works with following data types and connectors:
        #filter sd data frame
        data_sd_filtered = data_sd[(data_sd['type_base'].isin(filt_type_base))&(data_sd['inout'].isin(filt_inout))]
        
        #connect links $XXXXX->$XXXXX
        d = dict(zip(data_tdc['chart_block_signal'].values, data_tdc['value_linked'].values))
        data_sd_filtered['chart_block_signal'] = data_sd_filtered["chart"]+"/"+data_sd_filtered["block"]+"."+data_sd_filtered["signal"]
        
        data_sd_filtered['val_tdc'] = data_sd_filtered['chart_block_signal'].map(d)
        data_sd_filtered['compare_val_sd'] = data_sd_filtered.apply(lambda row: clean_sd_tdc_str_values(\
           row['value_linked'], row['val_tdc'],row['type_base'], row['type_conv'],row['chart_block_signal'],row['inout'],f), axis=1)
      
    if to_file:
        file_suffix = "@_compare_singals_sdtdc_%s"%(str(timestamp.date()))
        file_prefix = "report_@"
        try:
            # Create target Directory
            os.mkdir(os.path.join(PATH_TO_REPORTS))
        except FileExistsError:
            pass
        #sys.stdout = open(os.path.join(PATH_TO_REPORTS,(file_prefix+FILE_NAME_DATA1.split(".")[0]+file_suffix+".txt")), 'w')
        with open(os.path.join(PATH_TO_REPORTS,(file_prefix+SD_RACK_NAMES[0]+file_suffix+".txt")), 'w') as f: 
            generate_report(f)
    else:
        generate_report(sys.stdout)


In [120]:
###########EXECUTION CELL###########
#report find missing blocks
if(COMPARISON_TYPE==COMTYPE_SDSD):
    report_find_miss_blocks_sdsd(sd1_data_s2,sd2_data_s2,to_file = True)
elif  (COMPARISON_TYPE==COMTYPE_SDTDC):  
    report_find_miss_blocks_sdtdc(sd1_data_s2,tdc2_data_a1,to_file = True)
elif  (COMPARISON_TYPE==COMTYPE_TDCSD):  
    raise Exception('TDC to SD not implemented yet')
elif  (COMPARISON_TYPE==COMTYPE_TDCTDC):  
    raise Exception('TDC to TDC not implemented yet')

In [13]:
#sd1_data_s3[sd1_data_s3["block"]=="T250"]

In [121]:
###########EXECUTION CELL###########
#report find missing signals
if(COMPARISON_TYPE==COMTYPE_SDSD):
    report_comp_signals_sdsd(sd1_data_s3,sd2_data_s3,to_file = True)
elif  (COMPARISON_TYPE==COMTYPE_SDTDC):  
    report_comp_signals_sdtdc(sd1_data_s3,tdc2_data_a1,to_file = True)
elif  (COMPARISON_TYPE==COMTYPE_TDCSD):  
    raise Exception('TDC to SD not implemented yet')
elif  (COMPARISON_TYPE==COMTYPE_TDCTDC):  
    raise Exception('TDC to TDC not implemented yet')

In [48]:
int(float(re.sub('[^0-9.E-]','', '70.01')))

70

In [22]:
sd1_data_s3[(sd1_data_s3["value_linked"]=="PDX1_1\BLS_X.QS")]

,signal,type_base,inout,type_conv,value,cpu,cpu_source,cpu_link,chart,block,link,value_linked
49312,IS,V2,<,None,$BLS_X,PN-D05_P3,PN-D01_P1,PN-D01_P1_$BLS_X,PWR__3,BLS_X,[$BLS_X],PDX1_1\BLS_X.QS
51795,X,N2,<,V2,$BLS_X,PN-D01_P1,PN-D01_P1,PN-D01_P1_$BLS_X,QDAD_1,BLS_X,[$BLS_X],PDX1_1\BLS_X.QS


In [25]:
sd1_data_s3[(sd1_data_s3["chart"]=="BLOC_1")&(sd1_data_s3["block"]=="WT_EX2")]

,signal,type_base,inout,type_conv,value,cpu,cpu_source,cpu_link,chart,block,link,value_linked
179,X,NF,<,None,$WT_COI,PN-D01_P1,PN-D04_P2,PN-D04_P2_$WT_COI,BLOC_1,WT_EX2,[$WT_COI],PDW1_2\W10020.Y08
180,Y,NF,>,None,None,PN-D01_P1,PN-D01_P1,None,BLOC_1,WT_EX2,"[TCOIL1.X2, IBS100.X2]","TCOIL1.X2,IBS100.X2"


In [25]:
tdc2_data_a1[(tdc2_data_a1["chart"]=="TRAK_3")&(tdc2_data_a1["block"]=="FBBAF")]

,rack_cpu,chart,block,signal,value,link,type,out,rack,cpu,chartblock,chart_block_signal,value_linked
42770,SX01\D05_P3\,TRAK_3,FBBAF,N,5,NaN,INT,False,SX01,D05_P3,TRAK_3/FBBAF,TRAK_3/FBBAF.N,5
42771,SX01\D05_P3\,TRAK_3,FBBAF,S,NaN,DIS_ST.Q,DWORD,False,SX01,D05_P3,TRAK_3/FBBAF,TRAK_3/FBBAF.S,DIS_ST.Q
42772,SX01\D05_P3\,TRAK_3,FBBAF,FA,NaN,DIS_WO.Q,DWORD,False,SX01,D05_P3,TRAK_3/FBBAF,TRAK_3/FBBAF.FA,DIS_WO.Q
42773,SX01\D05_P3\,TRAK_3,FBBAF,FE,NaN,DIS_WC.Q,DWORD,False,SX01,D05_P3,TRAK_3/FBBAF,TRAK_3/FBBAF.FE,DIS_WC.Q
42774,SX01\D05_P3\,TRAK_3,FBBAF,AKI,NaN,TRAK_3\BAFACT.QS,WORD,False,SX01,D05_P3,TRAK_3/FBBAF,TRAK_3/FBBAF.AKI,TRAK_3\BAFACT.QS
42775,SX01\D05_P3\,TRAK_3,FBBAF,ZBI,NaN,TRAK_3\BAFZBI.QS,WORD,False,SX01,D05_P3,TRAK_3/FBBAF,TRAK_3/FBBAF.ZBI,TRAK_3\BAFZBI.QS
42776,SX01\D05_P3\,TRAK_3,FBBAF,BEI,NaN,TRAK_3\BAFBEI.QS,WORD,False,SX01,D05_P3,TRAK_3/FBBAF,TRAK_3/FBBAF.BEI,TRAK_3\BAFBEI.QS
42777,SX01\D05_P3\,TRAK_3,FBBAF,VB,NaN,TRAK_3\VSPACT.Q,DWORD,False,SX01,D05_P3,TRAK_3/FBBAF,TRAK_3/FBBAF.VB,TRAK_3\VSPACT.Q
42778,SX01\D05_P3\,TRAK_3,FBBAF,SET,NaN,TRAK_3\BAFSET.Q,BOOL,False,SX01,D05_P3,TRAK_3/FBBAF,TRAK_3/FBBAF.SET,TRAK_3\BAFSET.Q
42779,SX01\D05_P3\,TRAK_3,FBBAF,RES,NaN,TRAK_3\RQ_RST.Q,BOOL,False,SX01,D05_P3,TRAK_3/FBBAF,TRAK_3/FBBAF.RES,TRAK_3\RQ_RST.Q


In [90]:
sd1_data_s3[(sd1_data_s3["chart"]=="MOUSW1")&(sd1_data_s3["inout"]=="<")]

,signal,type_base,inout,type_conv,value,cpu,cpu_source,cpu_link,chart,block,link,value_linked
26126,IS,V2,<,None,$OSHH01,PN-D01_P1,PN-D01_P1,PN-D01_P1_$OSHH01,MOUSW1,OUSW01,[$OSHH01],MHHY_1\OSHH01.QS
26143,IS,V2,<,None,$OSHH02,PN-D01_P1,PN-D01_P1,PN-D01_P1_$OSHH02,MOUSW1,OUSW02,[$OSHH02],MHHY_1\OSHH02.QS
26160,IS,V2,<,None,$OSSH01,PN-D01_P1,PN-D01_P1,PN-D01_P1_$OSSH01,MOUSW1,OUSW03,[$OSSH01],MSHY_1\OSSH01.QS
26177,IS,V2,<,None,$OSSH02,PN-D01_P1,PN-D01_P1,PN-D01_P1_$OSSH02,MOUSW1,OUSW04,[$OSSH02],MSHY_1\OSSH02.QS
26194,IS,V2,<,None,$OSMO01,PN-D01_P1,PN-D01_P1,PN-D01_P1_$OSMO01,MOUSW1,OUSW05,[$OSMO01],MMOES1\OSMO01.QS
26211,I1,B1,<,None,OUSW01.Q1,PN-D01_P1,PN-D01_P1,None,MOUSW1,HD_OK,[],MOUSW1\OUSW01.Q1
26212,I2,B1,<,None,OUSW03.Q1,PN-D01_P1,PN-D01_P1,None,MOUSW1,HD_OK,[],MOUSW1\OUSW03.Q1
26213,I3,B1,<,None,1,PN-D01_P1,PN-D01_P1,None,MOUSW1,HD_OK,[],1
26214,I4,B1,<,None,1,PN-D01_P1,PN-D01_P1,None,MOUSW1,HD_OK,[],1
26216,I1,B1,<,None,OUSW06.Q3,PN-D01_P1,PN-D01_P1,None,MOUSW1,HD_OK2,[],MOUSW1\OUSW06.Q3
